## Probabilidades 

Referencia:

1 . [Probability and Statistics for Computer Scientists](https://www.crcpress.com/Probability-and-Statistics-for-Computer-Scientists-Second-Edition/Baron/9781439875902) de Michael Baron.

## Variables Aleatorias 

## La distribución Normal

Una de las distribuciones más importantes de la teoria de las Probabilidades es la `distribución normal`, determinada completamente determinada por dos parámetros: su media ($\mu$) y su desviación estándar ($\sigma$). La media indica donde la distribución es centrada y la desviación estándar cuán 'ancha' es. La distribución normal tiene la forma:

$$ 
\begin{equation}
f(x | \mu, \sigma) =  \frac{1}{\sqrt{2\pi}\sigma} exp\biggl(-\frac{(x - \mu)^2}{2\sigma^2}\biggr)$$
\end{equation}
Que se implementa en Python:

In [8]:
import math

def normal_pdf(x, mu=0, sigma=1):
    sqrt_two_pi = math.sqrt(2 * math.pi)
    return (math.exp(-(x-mu) ** 2 / 2 / sigma ** 2)/ (sqrt_two_pi * sigma))

Mostremos algunos gráficos de las funciones de densidad, para distintos valores de la media y desviación estándar:

In [2]:
from matplotlib import pyplot as plt

xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_pdf(x,sigma=1) for x in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_pdf(x,sigma=2) for x in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_pdf(x,sigma=0.5) for x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_pdf(x,mu=-1) for x in xs],'-.',label='mu=-1,sigma=1')
plt.legend()
plt.title("Varias funciones de densidad  para la distribucion Normal ")
plt.show()

Cuando $\mu = 0$ y $\sigma = 1$, la distribución normal se conoce como, **distribución normal estándar**. Si `Z` es una distribución normal estándar, entonces resulta que:

$$X = \sigma Z + \mu$$

es también normal, pero con media $\mu$ y desviación estándar $\sigma$. Por otro lado, si $X$ es una variable aleatoria normal con media  $\mu$ y desviación estándar $\sigma$,

$$Z = (X -\mu)/\sigma$$

es una variable normal estándar. La función de distribución acumulativa (CDF), no puede ser escrita de manera 'elemental', pero puede ser escrita usando `math.erf`, que retorna la [función error](https://en.wikipedia.org/wiki/Error_function):

$F(x) = \frac{1}{2}\biggl[1 + erf\biggl( \frac{x - \mu}{\sqrt{2}\sigma}\biggr)\biggr]$

In [9]:
import math
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

Otra vez mostremos los gráficos de  algunas  CDF:

In [10]:
from matplotlib import pyplot as plt

xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_cdf(x,sigma=1) for x in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_cdf(x,sigma=2) for x in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_cdf(x,sigma=0.5) for x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_cdf(x,mu=-1) for x in xs],'-.',label='mu=-1,sigma=1')
plt.legend(loc=4) 
plt.title("Varias  cdfs de la distribución normal")
plt.show()

A veces tendremos que invertir `normal_cdf` para encontrar el valor correspondiente a una probabilidad específica(funcion cuantil). No hay manera fácil de calcular su inversa, pero  `normal_cdf` es  continua y estrictamente creciente, por lo que podemos utilizar  `búsqueda binaria` *(Joel Grus-Data Science from Scratch: First Principles with Python)*

In [11]:
def inversa_normal_cdf(p, mu=0, sigma=1, tolerancia=0.00001):
    
    """encuentra aproximadamente la inversa de CDF usando busqueda binaria"""
    
    # si no es estándar, calculamos el estándar y reescalamos
    
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerancia=tolerancia)
    l_z, l_p = -10.0, 0 # normal_cdf(-10) es (muy cerca a) 0
    h_z, h_p = 10.0, 1 # normal_cdf(10) es (muy cerca a) 1
    
    while h_z - l_z > tolerancia:
        medio_z = (l_z + h_z) / 2 # consideramo el punto medio
        mid_p = normal_cdf(medio_z) # y el valor cdf's existe
        if medio_p < p:
            #  el punto medio es aun muy bajo, buscamos arriba 
            l_z, l_p = medio_z, medio_p
        elif medio_p > p:
            
            # el punto medio es muy alto, busco 
            h_z, h_p = medio_z, medio_p
        else:
            break
    return medio_z

La función repetidamente biseca un intervalo, hasta que se acerca en un Z que es lo suficiente cerca a la probabilidad deseada.

## Ley de los Grandes Números



## El teorema del Límite Central 

Una de las razones por la que la  distribución normal es tan útil, proviene del teorema del límite central, que
dice (en esencia) que una variable aleatoria definida como el promedio de un gran número de variables aleatorias independientes e idénticamente distribuidas es en sí misma aproximadamente  una distribución normal.


En particular, si $x_1,\dots , x_n $ son variables aleatorias con media $\mu$ y desviación estándar $\sigma$,
y si $n$ es grande, entonces:

$$\frac{1}{n}\bigl(x_1 + \cdots x_n\bigr)$$

es aproximadamente una distribución normal con media $\mu$ y desviación estándar  $\sigma /\sqrt{n}$. Equivalentemente:

$$\frac{\bigl(x_1 + \cdots +x_n \bigr)}{\sigma\sqrt{n}}$$

es aproximadamente una distribución normal con media `0` y desviación estándar `1`.

Una manera de ver este resultado, es a través de las variables aleatorias binomiales, las cuales tienen dos parámetros $n$ y $p$. Una variable aleatoria `Binomial(n,p)` es simplemente la suma de $n$ variables aleatorias Bernoulli independientes `Bernoulli(p)` cada uno de los cuales es igual a `1` con probabilidad `p` y `0` con probabilidad `1 - p`:

In [12]:
import random, collections 

def bernoulli_prueba(p):
    return 1 if random.random() < p else 0

def binomial(n,p):
    return sum(bernoulli_prueba(p) for _ in range(n))

La media de una variable `Bernoulli(p)` es $p$ y su desviación estándar es $\sqrt{p(1 -p)}$. El teorema del Límite Central dice que cuando $n$ es muy grande, una variable `Binomial(n ,p)` es aproximadamente una variable aleatoria normal con media $\mu = np$ y desviación estándar $\sigma = \sqrt{np(1 -p)}$. Veamos un gráfico que muestra este resultado:

In [13]:
import math, random
from collections import Counter
from matplotlib import pyplot as plt


def bernoulli_prueba(p):
    return 1 if random.random() < p else 0

def binomial(n,p):
    return sum(bernoulli_prueba(p) for _ in range(n))
    

def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2


def graf_histograma(p, n, num_puntos):
    data = [binomial(n, p) for _ in range(num_puntos)]

    # usamos un grafico de barras para mostrar las muestras binomiales
    histograma  = Counter(data)
    plt.bar([x - 0.4 for x in histograma .keys()],
            [v / num_puntos for v in histograma .values()],
            0.8,
            color='0.75')
    mu = p * n
    sigma = math.sqrt(n * p * (1 - p))
    # usamos un grafico de  lineas para mostrar una  aproximacion normal
    xs = range(min(data), max(data) + 1)
    ys = [normal_cdf(i + 0.5, mu, sigma) - normal_cdf(i - 0.5, mu, sigma)
          for i in xs]
    plt.plot(xs,ys)
    plt.title("Distribucion Binomial vs. Approximacion Normal")
    plt.show()
    
# graf_histograma(0.75, 100, 10000) 

[scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html) es módulo de la libreria científica Scipy que  contiene un gran número de distribuciones de probabilidad, así como una creciente biblioteca de funciones estadísticas:


[animation](http://yihui.name/animation/) es un paquete de R para crear y exportar animaciones a una variedad de formatos (HTML / JS, GIF, vídeo, PDF), así como una galería de animaciones estadísticas, como muestra el siguiente ejemplo, acerca del teorema del Límite Central de la documentación de animation de Yuihui Xie

```R
library(animation)

saveHTML({
  par(mar = c(3, 3, 1, 0.5), mgp = c(1.5, 0.5, 0), tcl = -0.3)
  ani.options(interval = 0.1, nmax = ifelse(interactive(), 150, 10))
  clt.ani(type = "h")
}, img.name = "clt.ani", htmlfile = "clt.ani.html", ani.height = 500,
ani.width = 600, title = "Demonstracion del teorema del Limite Central",
description = c("Esta animación muestra la distribución de la muestra ",
                  " significa que el tamaño de la muestra crece."))
```